In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# make the notebook executable on google colab
if 'google.colab' in str(get_ipython()):
    # install package
    try:
      import gsemantique
    except ImportError:
      !git clone https://github.com/fkroeber/gsemantique.git
      !cd gsemantique && pip install .
    # change working directory
    demo_dir = "gsemantique/demo"
    import os
    if not os.getcwd().endswith(demo_dir):
      os.chdir(demo_dir)

# get data finder logger if script is not run in colab
else:
    import logging
    logger = logging.getLogger("gsemantique.data.search")
    logger.setLevel(logging.INFO)
    stream_handler = logging.StreamHandler()
    logger.addHandler(stream_handler)

In [3]:
# import semantique & gsemantique
import semantique as sq
import gsemantique as gsq

# some other packages
import geopandas as gpd
import json
import os
import pandas as pd
import planetary_computer as pc
import pystac
import shutil
from pystac_client import Client
from shapely.geometry import box

## Investigating existing datasets

Gsemantique offers preconfigured access to a total of 11 different datasets that are provided with global (or near global) coverage. The datasets originate from 3 different endpoints (and the Sentinel-1 and Sentinel-2 data sets can be retrieved from two of these providers). The different datasets include the following types of data:  
* original reflectance values (S1, S2, Landsat)
* terrain/topography products (DEM, DSM) 
* hydrological products (HAND, GSW) 
* burned area products (BAM, BAD) 
* landcover products (ESRI, ESA)

Internally, all `Dataset` instances are organised in a `DatasetCatalog` class. An empty `DatasetCatalog` can be imported and instantiated as shown below. The `.load()` method needs to be called to populate the catalog with the pre-defined data products listed above.

In [4]:
ds_catalog = gsq.DatasetCatalog()
ds_catalog.load()

To inspect the data catalog, you can use the following commands.

In [5]:
print(ds_catalog)

DatasetCatalog containing
- 3 providers (catalogs)
- 13 datasets (collections)
- 71 layers (assets)

Datasets:
     provider                   collection        category temporality
0   Planetary               sentinel-1-rtc             SAR           s
1   Planetary               sentinel-2-l2a   multispectral           s
2   Planetary                landsat-c2-l2   multispectral           s
3   Planetary               esa-worldcover       landcover           Y
4   Planetary           io-lulc-annual-v02       landcover           Y
5   Planetary                      nasadem             DEM        None
6   Planetary               cop-dem-glo-30             DSM        None
7   Planetary               modis-64A1-061  fire detection           M
8   Planetary               modis-14A2-061  fire detection           D
9   Planetary                      jrc-gsw  hydrogeography        None
10  Element84               sentinel-2-l2a   multispectral           s
11        ASF  sentinel-1-global-cohe

In [6]:
ds_catalog.parse_as_table(keys=None)

,category,collection,copyright,endpoint,info,layout_bands,layout_keys,provider,spatial_extent,src,temporal_extent,temporality,n_bands
0,SAR,sentinel-1-rtc,CC BY 4.0,https://planetarycomputer.microsoft.com/api/st...,Sentinel-1 represent radar imaging (SAR) satel...,"{'s1_amp_vv': {'name': 'vv', 'description': 'G...","[(Planetary, reflectance, s1_amp_vv), (Planeta...",Planetary,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2014-10-10 00:28:21+00:00, None]",s,4
1,multispectral,sentinel-2-l2a,Copernicus Sentinel Data Terms,https://planetarycomputer.microsoft.com/api/st...,The Sentinel-2 program provides global imagery...,"{'s2_band01': {'name': 'B01', 'description': '...","[(Planetary, reflectance, s2_band01), (Planeta...",Planetary,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2015-06-27 10:25:31+00:00, None]",s,13
2,multispectral,landsat-c2-l2,Public Domain (https://www.usgs.gov/emergency-...,https://planetarycomputer.microsoft.com/api/st...,"Landsat Collection 2 Level-2 Science Products,...","{'lndst_coastal': {'name': 'coastal', 'descrip...","[(Planetary, reflectance, lndst_coastal), (Pla...",Planetary,"[-180.0, -90.0, 180.0, 90.0]",https://planetarycomputer.microsoft.com/datase...,"[1982-08-22 00:00:00+00:00, None]",s,10
3,landcover,esa-worldcover,Creative Commons Attribution 4.0 International...,https://planetarycomputer.microsoft.com/api/st...,The European Space Agency (ESA) WorldCover pro...,"{'esa_lc': {'name': 'map', 'description': 'ESA...","[(Planetary, classification, esa_lc)]",Planetary,"[-180.0, -60.0, 180.0, 83.0]",https://planetarycomputer.microsoft.com/datase...,"[2020-01-01 00:00:00+00:00, 2021-12-31 23:59:5...",Y,1
4,landcover,io-lulc-annual-v02,Creative Commons BY-4.0,https://planetarycomputer.microsoft.com/api/st...,Time series of annual global maps of land use ...,"{'impact_lc': {'name': 'data', 'description': ...","[(Planetary, classification, impact_lc)]",Planetary,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2017-01-01 00:00:00+00:00, 2024-01-01 00:00:0...",Y,1
5,DEM,nasadem,Public Domain (https://lpdaac.usgs.gov/data/da...,https://planetarycomputer.microsoft.com/api/st...,NASADEM provides global topographic data at 1 ...,"{'dem': {'name': 'elevation', 'description': '...","[(Planetary, topography, dem)]",Planetary,"[-179.0, -56.0, 179.0, 61.0]",https://planetarycomputer.microsoft.com/datase...,"[2000-02-20 00:00:00+00:00, 2000-02-20 00:00:0...",None,1
6,DSM,cop-dem-glo-30,None,https://planetarycomputer.microsoft.com/api/st...,The Copernicus DEM is a digital surface model ...,"{'dsm': {'name': 'data', 'description': 'Digit...","[(Planetary, topography, dsm)]",Planetary,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2021-04-22 00:00:00+00:00, 2021-04-22 00:00:0...",None,1
7,fire detection,modis-64A1-061,None,https://planetarycomputer.microsoft.com/api/st...,The Terra and Aqua combined MCD64A1 Version 6....,"{'m_burn_date': {'name': 'Burn_Date', 'descrip...","[(Planetary, burned_mapping, m_burn_date), (Pl...",Planetary,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2000-11-01 00:00:00+00:00, None]",M,3
8,fire detection,modis-14A2-061,None,https://planetarycomputer.microsoft.com/api/st...,The Moderate Resolution Imaging Spectroradiome...,"{'w_burn_qa': {'name': 'QA', 'description': 'P...","[(Planetary, burned_mapping, w_burn_qa), (Plan...",Planetary,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2000-02-18 00:00:00+00:00, None]",D,2
9,hydrogeography,jrc-gsw,Copernicus Open Access Policy,https://planetarycomputer.microsoft.com/api/st...,Global surface water products from the Europea...,"{'change': {'name': 'change', 'description': '...","[(Planetary, hydrogeography, change), (Planeta...",Planetary,"[-180.0, -56.0, 180.0, 78.0]",https://planetarycomputer.microsoft.com/datase...,"[1984-03-01 00:00:00+00:00, 2020-12-31 11:59:5...",None,4


With the `Dataset` instance, a core set of high-level overview information. To search for and work with a data set, more detailed information on its different data layers and their charactersitics is needed. Following the conceptual framework of semantique, this information is captured in a so-called dataset layout. To quote from the documentation of [semantique](https://zgis.github.io/semantique/_notebooks/recipes.html#Representing-an-EO-data-cube):

>In semantique, an EO data cube is always represented by its layout. The layout is a JSON-structured file containing a metadata object for each data layer in the cube. The software expert should distribute it alongside the EO data cube, such that the application expert can use it to initialize a Python object that represents the cube.

The layout file for the pre-configured datasets listed above is part of the gsemantique package (see [here](../gsemantique/data/layout.json), accessible via gsq.LAYOUT_PATH). It is organised hierarchically with dataset providers as top-level keys, e.g. (Planet, reflectance, s1_amp_vv). The layout information is bound to `Dataset` instances with the layout-specific attributes "layout_file", "layout_keys", "layout_bands". The "layout_bands" information is directly parsed from the layout file.

In [7]:
layout_bands = ds_catalog.filter(collection="landsat-c2-l2")["layout_bands"].iloc[0]
layout_bands

{'lndst_coastal': {'name': 'coastal',
  'description': 'Coastal with following specs: Spatial resolution: 30 m. Center wavelength: 0.44 Î¼m Î¼m (OLI). Full width half max: 0.02 Î¼m (OLI). Note: Only available for OLI (i.e. Landsat 8/9 from 2013 onwards). Scale: 0.0000275. Offset -0.2.',
  'type': 'continuous',
  'values': {},
  'dtype': 'uint16',
  'na_value': 'None',
  'reference': ['Planetary', 'reflectance', 'lndst_coastal']},
 'lndst_blue': {'name': 'blue',
  'description': 'Visible blue with following specs: Spatial resolution: 30 m. Center wavelength: 0.49 Î¼m (TM), 0.48 Î¼m (ETM), 0.48 Î¼m (OLI). Full width half max: 0.07 Î¼m (TM), 0.07 Î¼m (ETM), 0.06 Î¼m (OLI). Scale: 0.0000275. Offset -0.2.',
  'type': 'continuous',
  'values': {},
  'dtype': 'uint16',
  'na_value': 'None',
  'reference': ['Planetary', 'reflectance', 'lndst_blue']},
 'lndst_green': {'name': 'green',
  'description': 'Visible green with following specs: Spatial resolution: 30 m. Center wavelength: 0.56 Î¼m (TM

## Searching for data

In order to build on-demand STACCubes, one needs to have a list of items at hand that are to be ingested into the cube. To this end, semantique offers the `Finder` class. Its initialisation requires a dataset catalog and the spatio-temporal extent of interest. Subsequently, you can call its .search_man() method to perform a pystac-client based search for items based on a given layer_key. The results are then captured in the property item_coll as shown below. 

In [8]:
?gsq.Finder

Init signature:
gsq.Finder(
    ds_catalog,
    t_start,
    t_end,
    aoi,
    layout_file='c:\\users\\felix\\repositories\\gsemantique\\gsemantique\\data\\layout.json',
)
Docstring:     
Searches a given catalog to output the item's STAC metadata
This can then be forwarded to STACCube.
Init docstring:
Initializes the Finder object with the specified inputs

Args:
    ds_catalog (DatasetCatalog): Dataset catalog containing the data sets to be searched
    t_start (str): Start time of the search
    t_end (str): End time of the search
    aoi (shapely.geometry): Area of interest in WGS84 coordinates (EPSG:4326)
    layout_file (str): Path to the datacube layout file
File:           c:\users\felix\repositories\gsemantique\gsemantique\data\search.py
Type:           type
Subclasses:     

In [9]:
t_start, t_end = "2019-03-20", "2019-03-31"
xmin, ymin, xmax, ymax = 9.2, 50.13, 9.25, 50.18
aoi = box(xmin, ymin, xmax, ymax)
layer_key = ("Planetary", "reflectance", "lndst_qa")

fdr = gsq.Finder(ds_catalog, t_start, t_end, aoi)
fdr.search_man(layer_key)
fdr.item_coll

Initialise search for ('Planetary', 'reflectance', 'lndst_qa')
Search started
Search finished
Search postprocessed
Found 2 datasets


Using the manual search function, requires to have a layer_key, i.e. a data layout specific attribute. As a user, you usually want to create a recipe on the semantic level. Additionally, you do have access to some mapping of entities and the layout file (both of which were possibly provided to you in a preconfigured manner by an EO expert). You most certainly don't care about the layer key but simply want to have the data needed for you recipe being automatically searched for. To this end, the Finder has a second method called .search_auto. This method takes a recipe and mapping as an input and automatically resolves the data layer references needed to evaluate the recipe. The search results are again stored in the .item_coll attribute.

In [10]:
# define an empty data cube
with open(gsq.LAYOUT_PATH, "r") as file:
    dc = sq.datacube.STACCube(
        json.load(file), 
        src=[]
    )

# define exemplary mapping
# sth created by yourself or given to you by a EO expert
mapping = sq.mapping.Semantique()
mapping["entity"] = {}
valid_indices = [
    dc.layout["Planetary"]["reflectance"]["lndst_qa"]["labels"][x]
    for x in ["clear_land_etm", "clear_water_etm", "clear_land_oli", "clear_water_oli"]
]
mapping["entity"]["cloudfree"] = {
    "color": sq.layer("Planetary", "reflectance", "lndst_qa").evaluate("in", valid_indices)
}
mapping["entity"]["water"] = {
    "color": (
        sq.layer("Planetary", "reflectance", "lndst_green")
            .evaluate("normalized_difference", sq.layer("Planetary", "reflectance", "lndst_nir08"))
            .evaluate("greater_equal", 0.2)
    )
}
mapping["entity"]["vegetation"] = {
    "color": (
        sq.layer("Planetary", "reflectance", "lndst_nir08")
            .evaluate("normalized_difference", sq.layer("Planetary", "reflectance", "lndst_red"))
            .evaluate("greater_equal", 0.2)
    )
}

# define exemplary recipe
recipe = sq.QueryRecipe()
recipe["cloudfree"] = (
    sq.entity("cloudfree")
    .reduce("count", "time")
)
recipe["water_frequency"] = (
    sq.entity("water")
    .filter(sq.entity("cloudfree"))
    .reduce("percentage", "time")
)

fdr.search_auto(recipe, mapping)
fdr.item_coll

The recipe references the following data layers:
('Planetary', 'reflectance', 'lndst_nir08')
('Planetary', 'reflectance', 'lndst_green')
('Planetary', 'reflectance', 'lndst_qa')
Initialise search for ('Planetary', 'reflectance', 'lndst_nir08')
Search started


Search finished
Search postprocessed
Found 2 datasets
Initialise search for ('Planetary', 'reflectance', 'lndst_green')
Search started
Search finished
Search postprocessed
Found 2 datasets
Initialise search for ('Planetary', 'reflectance', 'lndst_qa')
Search started
Search finished
Search postprocessed
Found 2 datasets


Note that items will be retrieved multiple times, if the recipe references different assets of the same item. The results are then consolidated automatically by merging assets on an item level. In the example above, the three different assets all belong to items from the landsat collection.

## Downloading data 

For reason such as data sovereignty, performance improvements for repeated executions or the ability to explore the input data in common GIS systems it may be desirable not to pull the input data for the STACCube from the network for each recipe evaluation, but to persist the data locally. gsemantique offers the `Downloader` class for this purpose, which downloads the results of the `Finder` and generates a corresponding STAC-conformant ItemCollection with the local asset pointers. Specifically, you can instantiate a `Downloader` by forwarding a Finder result to it. Calling the .run() method will start the download process and provide some indication of its progress. You can specify the output directory if desired.

In [11]:
?gsq.Downloader

Init signature: gsq.Downloader(item_coll, out_dir=None, **kwargs)
Docstring:      <no docstring>
Init docstring:
Downloader class tailored to the needs of handling single &
multi-collection outputs as generated by the Finder. Results will
be downloaded & a STAC-conformant metadata description (catalog,
collection and item level) with corresponding relative local links
will be established.

Note: The created STAC metadata json are containing all the originally
provided information of the input item_coll, incl. for example the extra
asset field "semantique:key". Apart from the updated links, the STAC
metadata json is therefore equivalent to the input item_coll.

Args:
    item_coll (pystac.ItemCollection or list of pystac.item.Item):
        The Finder result (search.py) to be downloaded.
    out_dir (str): The directory to download the files to. If not specified,
        a new directory will be created with the current timestamp.
    **kwargs (dict): Keyword arguments forwarded to _STAC

In [12]:
item_coll = fdr.item_coll
out_dir = "files/data"

dwn = gsq.Downloader(item_coll, out_dir)
dwn.run()

landsat-c2-l2 (collection 1/1)
Download loop 0
Not enough items to estimate size. Skipping preview run.


Downloaded items: 2/2
Success rate: 100.00%



But what happens if you adapt or extend your recipe in a way that additional data layers that have not been downloaded are referenced now? You can simply forward the novel `Finder` result to the `Downloader` and specify the existing data directory. This way, the new data layers / assets will be added without re-downloading the existing ones. The locally saved STAC metadata json will be updated correspondingly.   

In the example below, the existing recipe is extended by an entity "vegetation" which references the RED band additionally to the NIR band that has already been downloaded before.      

In [13]:
# adapted recipe counting vegetation instead of water
recipe = sq.QueryRecipe()
recipe["cloudfree"] = (
    sq.entity("cloudfree")
    .reduce("count", "time")
)
recipe["vegetation_frequency"] = (
    sq.entity("vegetation")
    .filter(sq.entity("cloudfree"))
    .reduce("percentage", "time")
)

# re-run finder
fdr.search_auto(recipe, mapping)
fdr.item_coll

# re-run download
dwn = gsq.Downloader(fdr.item_coll, out_dir)
dwn.run()

The recipe references the following data layers:
('Planetary', 'reflectance', 'lndst_nir08')
('Planetary', 'reflectance', 'lndst_red')
('Planetary', 'reflectance', 'lndst_qa')
Initialise search for ('Planetary', 'reflectance', 'lndst_nir08')
Search started
Search finished
Search postprocessed
Found 2 datasets
Initialise search for ('Planetary', 'reflectance', 'lndst_red')
Search started
Search finished
Search postprocessed
Found 2 datasets
Initialise search for ('Planetary', 'reflectance', 'lndst_qa')
Search started
Search finished
Search postprocessed
Found 2 datasets


landsat-c2-l2 (collection 1/1)
Download loop 0
Not enough items to estimate size. Skipping preview run.


Downloaded items: 2/2
Success rate: 100.00%



Note that the local catalog contains relative links, such that shifting the data directory to other places doesn't break the links. If you want to re-read the item collection from the disk to use it for further processing, you need to make the references absolute again as shown below. 

In [14]:
# re-read requires to convert files to absolute references
catalog = pystac.Catalog.from_file(os.path.join(out_dir, "catalog.json"))
catalog.make_all_asset_hrefs_absolute()
item_list = [x for x in catalog.get_items(recursive=True)]
local_coll = pystac.ItemCollection(item_list)
local_coll

In [15]:
# clean-up
shutil.rmtree(out_dir)

## Accessing data

The search result of items can be forwarded to the STACCube constructor as described in the [semantique documentation](https://zgis.github.io/semantique/_notebooks/datacube.html#The-STACCube-configuration). Note that within gsemantique data access is abstracted from the user, i.e. once a model is specified the code below will be executed internally. For these and other processing-related things, please find the basics_processing.ipynb.  

In [16]:
# construct STACCube with relevant items
with open(gsq.LAYOUT_PATH, "r") as file:
    dc = sq.datacube.STACCube(
        json.load(file), 
        src = item_coll,
        group_by_solar_day=True,
        dask_params=None,
    )

# define spatio-temporal extent
res = 0.001
epsg = 4326
time = sq.TemporalExtent(pd.Timestamp(fdr.params_search["t_start"]), pd.Timestamp(fdr.params_search["t_end"]))
space = sq.SpatialExtent(gpd.GeoDataFrame(geometry=[fdr.params_search["aoi"]], crs="EPSG:4326"))
extent = sq.processor.utils.parse_extent(space, time, spatial_resolution = [-res, res], crs = epsg)

# load data via retrieve 
data = dc.retrieve(*('Planetary', 'reflectance', 'lndst_nir08'), extent=extent)
data

c:\Users\felix\.virtualenvs\gsemantique\lib\site-packages\stackstac\prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


<xarray.DataArray 'nir08' (time: 2, y: 50, x: 50)> Size: 20kB
array([[[35082., 35335., 36327., ..., 36733., 37434., 35556.],
        [37003., 36458., 36069., ..., 35731., 35036., 35378.],
        [35560., 34565., 33515., ..., 37160., 37364., 38972.],
        ...,
        [37041., 35132., 34028., ..., 37763., 38687., 39690.],
        [35789., 34650., 34572., ..., 38899., 39158., 39812.],
        [35304., 34926., 35232., ..., 39801., 39649., 39654.]],

       [[   nan,    nan,    nan, ..., 25838., 26957., 28123.],
        [   nan,    nan,    nan, ..., 26776., 27649., 28838.],
        [   nan,    nan,    nan, ..., 27547., 27679., 28331.],
        ...,
        [   nan,    nan,    nan, ..., 21527., 22679., 22543.],
        [   nan,    nan,    nan, ..., 22574., 22535., 22942.],
        [   nan,    nan,    nan, ..., 22955., 23121., 23749.]]],
      dtype=float32)
Coordinates:
  * x              (x) float64 400B 9.2 9.201 9.203 9.204 ... 9.248 9.248 9.25
  * y              (y) float64 400B 50.18 50.18 50.18 ... 50.13 50.13 50.13
    temporal_ref   int32 4B 0
  * time           (time) datetime64[ns] 16B 2019-03-26 2019-03-27
    spatial_feats  (y, x) float64 20kB 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
Attributes:
    spec:        RasterSpec(epsg=4326, bounds=(9.2, 50.13, 9.25, 50.18), reso...
    crs:         epsg:4326
    transform:   | 0.00, 0.00, 9.20|\n| 0.00,-0.00, 50.18|\n| 0.00, 0.00, 1.00|
    resolution:  0.001
    value_type:  continuous

## Adding new datasets

### General workflow

Gsemantique is set up to be expandable such that data sets beyond the pre-configured content of the existing dataset catalog can be used. The process of how to add new data sets to the catalog is described below.

First, an instance of the `Dataset` class needs to be created. Data product characteristics can be manually entered when doing so. The minimum set of parameters that needs to be specified includes "provider", "endpoint" and "collection" and "temporality". If not specified, the spatio-temporal extent will be automatically inferred by querying the corresponding STAC properties of the collection. Even though not strictly needed, it is advisable to specify the remaining attributes ("category", "src", "info" and "copyright").

In [17]:
ds = gsq.Dataset(
    provider="Planetary",
    endpoint="https://planetarycomputer.microsoft.com/api/stac/v1",
    collection="alos-fnf-mosaic",
    temporality="Y",
    temporal_extent=None,
    spatial_extent=None,
    category="SAR",
    src="https://planetarycomputer.microsoft.com/dataset/alos-fnf-mosaic",
    info="Global forest/non-forest maps  (FNF) based on 25m resolution SAR datasets generated by JAXA using the L-band Synthetic Aperture Radar sensors on the Advanced Land Observing Satellite-2 (ALOS-2 PALSAR-2), the Advanced Land Observing Satellite (ALOS PALSAR) and the Japanese Earth Resources Satellite-1 (JERS-1 SAR). FNF maps were generated by a Random Forest machine learning-based classification, Forest is defined as the tree covered land with an area larger than 0.5 ha and a canopy cover of over 10 %, in accordance with the FAO definition of forest.",
    copyright="Jaxa terms of use of research data",
)

As explained in the first section of this notebook, Dataset instances summarise information on the collection level whereas the data layer specific information needs to be added in the layout.json. Referencable assets need to be entered there with their characteristics as described in the [semantique documentation](https://zgis.github.io/semantique/_notebooks/datacube.html#The-STACCube-configuration). If the structure of the data at hand is unknown, an exemplary request can be carried out as shown below to investigate the available assets before entering them in the layout.json.

In [18]:
# make an exemplary search to investigate the structure of the data
catalog = Client.open(ds.endpoint, modifier=pc.sign_inplace)
query = catalog.search(
    collections=ds.collection,
    datetime=ds.temporal_extent,
    max_items=1,
)
item_coll = query.item_collection()
item_coll[0]

<Item id=S56W072_20_FNF>

As you can see from the search result above, there is one relevant data layer named "C". This data layer contains the classification forest vs. non-forest with a total of 5 classes. The corresponding information can also been drawn from the [original data documentation](https://planetarycomputer.microsoft.com/dataset/alos-fnf-mosaic). Along with dtype, this information needs to be entered to extend the existing layout.json file, which can then be saved somewhere to the disk. In our example, the extended layout looks like [this](files/layout_ext.json). The layout reference key - here (Planet, classification, forest) - can be choosen freely. Adhering to the organisation of the existing catalog, the dataset provider (Planetary) is used as top-level key to avoid ambiguities in cases where the same data layer may be offered by different providers.

Next, the connection between the Dataset instance and novel layout parts is established by providing the attributes layout file and layout keys during an .add_layout_info call. The dataset instance can then be added to the overall data catalog.

In [19]:
ds.add_layout_info(
    keys = [("Planetary", "classification", "forest")],
    file = os.path.abspath("files/layout_ext.json")
)
ds_catalog.add(ds)

To persist the data set as part of the data catalogue for future use you can call the .save() method and reload it from the pickled file next time. Per default, the data catalog is retrieved from a pickled file stored as part of the package instead. Its advisable to leave the later unaltered and save the new/extended data catalogue in a different location.

In [20]:
# persist data catalog locally
ds_catalog.save("files/new_catalog.pkl")

# re-load & show extended data catalog
ds_catalog = gsq.DatasetCatalog()
ds_catalog.load(cache_path="files/new_catalog.pkl")
ds_catalog.parse_as_table(keys=None)

,category,collection,copyright,endpoint,info,layout_bands,layout_keys,provider,spatial_extent,src,temporal_extent,temporality,n_bands
0,SAR,sentinel-1-rtc,CC BY 4.0,https://planetarycomputer.microsoft.com/api/st...,Sentinel-1 represent radar imaging (SAR) satel...,"{'s1_amp_vv': {'name': 'vv', 'description': 'G...","[(Planetary, reflectance, s1_amp_vv), (Planeta...",Planetary,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2014-10-10 00:28:21+00:00, None]",s,4
1,multispectral,sentinel-2-l2a,Copernicus Sentinel Data Terms,https://planetarycomputer.microsoft.com/api/st...,The Sentinel-2 program provides global imagery...,"{'s2_band01': {'name': 'B01', 'description': '...","[(Planetary, reflectance, s2_band01), (Planeta...",Planetary,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2015-06-27 10:25:31+00:00, None]",s,13
2,multispectral,landsat-c2-l2,Public Domain (https://www.usgs.gov/emergency-...,https://planetarycomputer.microsoft.com/api/st...,"Landsat Collection 2 Level-2 Science Products,...","{'lndst_coastal': {'name': 'coastal', 'descrip...","[(Planetary, reflectance, lndst_coastal), (Pla...",Planetary,"[-180.0, -90.0, 180.0, 90.0]",https://planetarycomputer.microsoft.com/datase...,"[1982-08-22 00:00:00+00:00, None]",s,10
3,landcover,esa-worldcover,Creative Commons Attribution 4.0 International...,https://planetarycomputer.microsoft.com/api/st...,The European Space Agency (ESA) WorldCover pro...,"{'esa_lc': {'name': 'map', 'description': 'ESA...","[(Planetary, classification, esa_lc)]",Planetary,"[-180.0, -60.0, 180.0, 83.0]",https://planetarycomputer.microsoft.com/datase...,"[2020-01-01 00:00:00+00:00, 2021-12-31 23:59:5...",Y,1
4,landcover,io-lulc-annual-v02,Creative Commons BY-4.0,https://planetarycomputer.microsoft.com/api/st...,Time series of annual global maps of land use ...,"{'impact_lc': {'name': 'data', 'description': ...","[(Planetary, classification, impact_lc)]",Planetary,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2017-01-01 00:00:00+00:00, 2024-01-01 00:00:0...",Y,1
5,DEM,nasadem,Public Domain (https://lpdaac.usgs.gov/data/da...,https://planetarycomputer.microsoft.com/api/st...,NASADEM provides global topographic data at 1 ...,"{'dem': {'name': 'elevation', 'description': '...","[(Planetary, topography, dem)]",Planetary,"[-179.0, -56.0, 179.0, 61.0]",https://planetarycomputer.microsoft.com/datase...,"[2000-02-20 00:00:00+00:00, 2000-02-20 00:00:0...",None,1
6,DSM,cop-dem-glo-30,None,https://planetarycomputer.microsoft.com/api/st...,The Copernicus DEM is a digital surface model ...,"{'dsm': {'name': 'data', 'description': 'Digit...","[(Planetary, topography, dsm)]",Planetary,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2021-04-22 00:00:00+00:00, 2021-04-22 00:00:0...",None,1
7,fire detection,modis-64A1-061,None,https://planetarycomputer.microsoft.com/api/st...,The Terra and Aqua combined MCD64A1 Version 6....,"{'m_burn_date': {'name': 'Burn_Date', 'descrip...","[(Planetary, burned_mapping, m_burn_date), (Pl...",Planetary,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2000-11-01 00:00:00+00:00, None]",M,3
8,fire detection,modis-14A2-061,None,https://planetarycomputer.microsoft.com/api/st...,The Moderate Resolution Imaging Spectroradiome...,"{'w_burn_qa': {'name': 'QA', 'description': 'P...","[(Planetary, burned_mapping, w_burn_qa), (Plan...",Planetary,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2000-02-18 00:00:00+00:00, None]",D,2
9,hydrogeography,jrc-gsw,Copernicus Open Access Policy,https://planetarycomputer.microsoft.com/api/st...,Global surface water products from the Europea...,"{'change': {'name': 'change', 'description': '...","[(Planetary, hydrogeography, change), (Planeta...",Planetary,"[-180.0, -56.0, 180.0, 78.0]",https://planetarycomputer.microsoft.com/datase...,"[1984-03-01 00:00:00+00:00, 2020-12-31 11:59:5...",None,4


Finally, note that `STACCube` requires assets to be organised as single-band products, otherwise the stacking process will fail. Search results from non-conformant datasets with an atypical organisation of items/assets may need to be transformed before forwarding them to the `STACCube` constructor. An example for this is the SAR data set "sentinel-1-global-coherence" as contained in the pre-configured data catalog. Since data layers are organised as items instead of assets here, a custom postprocessing procedure is implemented for this data set in the [Finder](../gsemantique/data/search.py). For our forest dataset example, this is not needed as the data layer is already organised as an asset (adhering to the common standards of STAC data organisation). 

### Details on dynamic & static STAC 

The previous description for adding new datasets represents the usual workflow for adding dynamic STAC catalogs. A dynamic STAC catalog (STAC API) provides a RESTful interface that allows users to search, filter, and paginate STAC Items dynamically. However, gsemantique can also be used to query and process data indexed in static STAC catalogs. A static catalog is a simple collection of JSON files stored in a hierarchical structure without a queryable API. While dynamic catalogs enable more efficient discovery and retrieval of geospatial assets at scale, static catalogues are simpler in design and a popular choice for many data sets. To see the variety of dynamic and static catalogs available, visit [STAC Index](https://stacindex.org/catalogs).

Below the usage of a static catalog is exemplified. We use the example of adding quarterly [NDVI timeseries data](https://stacindex.org/catalogs#/) as part of the Open Environmental Data Cube. The workflow of adding a static catalog is very similar to the pervious one: Creating a `Dataset` object along with the layout file amendments, and adding it to the `DatasetCatalog`. The usage of static STAC catalog manifests itself in the specification of a .json file address as an endpoint when defining the `Dataset` (as opposed to specifying a RESTful server endpoint).

In [21]:
# define static data set
ds = gsq.Dataset(
    provider="OpenGeoHub",
    endpoint="https://s3.eu-central-1.wasabisys.com/stac/odse/catalog.json",
    collection="lcv_ndvi_landsat.glad.ard",
    temporality="quarterly",
    temporal_extent=None,
    spatial_extent=None,
    category="multispectral",
    src="https://stacindex.org/catalogs/eco-datacube#/Qmhh3JTAhBD4WnfYyRZMqZyLwFkCFPaL56fk45ffJRsSUtufyJeqDwMF",
    info="Temporal composites of NDVI based on GLAD Landsat ARD, considering four quarterly per year and a median value per quarterly. The images for each quarterly were selected removing the cloud and cloud shadow pixels (quality assessment band) and considering the same calendar dates for all the years.",
    copyright="CC-BY-4.0"
)

c:\Users\felix\.virtualenvs\gsemantique\lib\site-packages\pystac_client\client.py:190: NoConformsTo: Server does not advertise any conformance classes.
  warnings.warn(NoConformsTo())


In [22]:
# retrieve an exemplary item to investigate the structure of the data
catalog = pystac.Catalog.from_file(ds.endpoint)
collection = catalog.get_child(ds.collection)
item_list = [x for x in collection.get_items(recursive=True)]
item_coll = pystac.ItemCollection(item_list)
item_coll[0]

<Item id=lcv_ndvi_landsat.glad.ard_1999.12.02..2000.03.20>

In [23]:
# add layout information to Dataset
ds.add_layout_info(
    keys = [("OpenGeoHub", "reflectance", "ndvi")],
    file = os.path.abspath("files/layout_ext.json")
)
ds_catalog.add(ds)

In [24]:
# persist data catalog locally
ds_catalog.save("files/new_catalog.pkl")

# re-load & show extended data catalog
ds_catalog = gsq.DatasetCatalog()
ds_catalog.load(cache_path="files/new_catalog.pkl")
ds_catalog.parse_as_table(keys=None)

,category,collection,copyright,endpoint,info,layout_bands,layout_keys,provider,spatial_extent,src,temporal_extent,temporality,n_bands
0,SAR,sentinel-1-rtc,CC BY 4.0,https://planetarycomputer.microsoft.com/api/st...,Sentinel-1 represent radar imaging (SAR) satel...,"{'s1_amp_vv': {'name': 'vv', 'description': 'G...","[(Planetary, reflectance, s1_amp_vv), (Planeta...",Planetary,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2014-10-10 00:28:21+00:00, None]",s,4
1,multispectral,sentinel-2-l2a,Copernicus Sentinel Data Terms,https://planetarycomputer.microsoft.com/api/st...,The Sentinel-2 program provides global imagery...,"{'s2_band01': {'name': 'B01', 'description': '...","[(Planetary, reflectance, s2_band01), (Planeta...",Planetary,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2015-06-27 10:25:31+00:00, None]",s,13
2,multispectral,landsat-c2-l2,Public Domain (https://www.usgs.gov/emergency-...,https://planetarycomputer.microsoft.com/api/st...,"Landsat Collection 2 Level-2 Science Products,...","{'lndst_coastal': {'name': 'coastal', 'descrip...","[(Planetary, reflectance, lndst_coastal), (Pla...",Planetary,"[-180.0, -90.0, 180.0, 90.0]",https://planetarycomputer.microsoft.com/datase...,"[1982-08-22 00:00:00+00:00, None]",s,10
3,landcover,esa-worldcover,Creative Commons Attribution 4.0 International...,https://planetarycomputer.microsoft.com/api/st...,The European Space Agency (ESA) WorldCover pro...,"{'esa_lc': {'name': 'map', 'description': 'ESA...","[(Planetary, classification, esa_lc)]",Planetary,"[-180.0, -60.0, 180.0, 83.0]",https://planetarycomputer.microsoft.com/datase...,"[2020-01-01 00:00:00+00:00, 2021-12-31 23:59:5...",Y,1
4,landcover,io-lulc-annual-v02,Creative Commons BY-4.0,https://planetarycomputer.microsoft.com/api/st...,Time series of annual global maps of land use ...,"{'impact_lc': {'name': 'data', 'description': ...","[(Planetary, classification, impact_lc)]",Planetary,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2017-01-01 00:00:00+00:00, 2024-01-01 00:00:0...",Y,1
5,DEM,nasadem,Public Domain (https://lpdaac.usgs.gov/data/da...,https://planetarycomputer.microsoft.com/api/st...,NASADEM provides global topographic data at 1 ...,"{'dem': {'name': 'elevation', 'description': '...","[(Planetary, topography, dem)]",Planetary,"[-179.0, -56.0, 179.0, 61.0]",https://planetarycomputer.microsoft.com/datase...,"[2000-02-20 00:00:00+00:00, 2000-02-20 00:00:0...",None,1
6,DSM,cop-dem-glo-30,None,https://planetarycomputer.microsoft.com/api/st...,The Copernicus DEM is a digital surface model ...,"{'dsm': {'name': 'data', 'description': 'Digit...","[(Planetary, topography, dsm)]",Planetary,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2021-04-22 00:00:00+00:00, 2021-04-22 00:00:0...",None,1
7,fire detection,modis-64A1-061,None,https://planetarycomputer.microsoft.com/api/st...,The Terra and Aqua combined MCD64A1 Version 6....,"{'m_burn_date': {'name': 'Burn_Date', 'descrip...","[(Planetary, burned_mapping, m_burn_date), (Pl...",Planetary,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2000-11-01 00:00:00+00:00, None]",M,3
8,fire detection,modis-14A2-061,None,https://planetarycomputer.microsoft.com/api/st...,The Moderate Resolution Imaging Spectroradiome...,"{'w_burn_qa': {'name': 'QA', 'description': 'P...","[(Planetary, burned_mapping, w_burn_qa), (Plan...",Planetary,"[-180, -90, 180, 90]",https://planetarycomputer.microsoft.com/datase...,"[2000-02-18 00:00:00+00:00, None]",D,2
9,hydrogeography,jrc-gsw,Copernicus Open Access Policy,https://planetarycomputer.microsoft.com/api/st...,Global surface water products from the Europea...,"{'change': {'name': 'change', 'description': '...","[(Planetary, hydrogeography, change), (Planeta...",Planetary,"[-180.0, -56.0, 180.0, 78.0]",https://planetarycomputer.microsoft.com/datase...,"[1984-03-01 00:00:00+00:00, 2020-12-31 11:59:5...",None,4


### Data search & access 

Once added to the `DatasetCatalog`, data from custom dynamic and/or static STAC catalogs can be used in the same way as all pre-defined datasets.

In [25]:
# demonstrate gsemantique's search & data retrieval functionality
layout_path = "files/layout_ext.json"
layer_key = ("OpenGeoHub", "reflectance", "ndvi")
fdr = gsq.Finder(ds_catalog, t_start, t_end, aoi, layout_path)
fdr.search_man(layer_key)

Initialise search for ('OpenGeoHub', 'reflectance', 'ndvi')
Search started
Static catalog is queried. Slow performance expected.
Search finished
Search postprocessed
Found 1 datasets


In [26]:
# construct STACCube with relevant items
with open(layout_path, "r") as file:
    dc = sq.datacube.STACCube(
        json.load(file), 
        src = fdr.item_coll,
        group_by_solar_day=True,
        dask_params=None,
    )

# define spatio-temporal extent
res = 0.001
epsg = 4326
time = sq.TemporalExtent(pd.Timestamp(fdr.params_search["t_start"]), pd.Timestamp(fdr.params_search["t_end"]))
space = sq.SpatialExtent(gpd.GeoDataFrame(geometry=[fdr.params_search["aoi"]], crs="EPSG:4326"))
extent = sq.processor.utils.parse_extent(space, time, spatial_resolution = [-res, res], crs = epsg)

# load data via retrieve 
data = dc.retrieve(*layer_key, extent=extent)
data

c:\Users\felix\.virtualenvs\gsemantique\lib\site-packages\stackstac\prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


<xarray.DataArray 'ndvi_landsat.glad.ard_p50_0..0cm' (time: 1, y: 50, x: 50)> Size: 10kB
array([[[141., 124., 137., ..., 185., 184., 183.],
        [145., 149., 154., ..., 180., 182., 182.],
        [128., 166., 175., ..., 177., 178., 178.],
        ...,
        [178., 178., 177., ..., 147., 161., 175.],
        [177., 178., 177., ..., 171., 170., 172.],
        [177., 175., 175., ..., 179., 176., 176.]]], dtype=float32)
Coordinates:
  * x              (x) float64 400B 9.2 9.201 9.203 9.204 ... 9.248 9.248 9.25
  * y              (y) float64 400B 50.18 50.18 50.18 ... 50.13 50.13 50.13
    temporal_ref   int32 4B 0
  * time           (time) datetime64[ns] 8B 2019-03-21
    spatial_feats  (y, x) float64 20kB 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
Attributes:
    spec:        RasterSpec(epsg=4326, bounds=(9.2, 50.13, 9.25, 50.18), reso...
    crs:         epsg:4326
    transform:   | 0.00, 0.00, 9.20|\n| 0.00,-0.00, 50.18|\n| 0.00, 0.00, 1.00|
    resolution:  0.001
    value_type:  continuous